In [136]:
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.output_parsers.list import CommaSeparatedListOutputParser
import re

In [137]:
from typing import List
from pydantic import BaseModel

class Item(BaseModel):
    artikel_id: int
    artikel_name: str
    anzahl: str
    preis_pro_menge: str
    gesamtpreis: str
    herkunft: str

class ParsedDocument(BaseModel):
    items: List[Item]

In [138]:
CONVERT_TO_CSV_TEMPLATE = """
Du bist ein Bot, der aus einer Einkaufsliste ein strukturiertes CSV macht.

Das CSV besteht aus unterschiedlichen Spalten:

* artikel_id
* artikel_name
* anzahl
* preis_pro_menge
* gesamtpreis
* herkunft

Außer items darf das CSV keine weiteren Elemente haben.
Es ist wichtig, dass jede gültige Position der Eingabe verarbeitet wird.
Wenn die Position nicht gültig ist, darf sie nicht in das CSV Resultat einfließen.
Gib das CSV Resultat so kompakt wie möglich aus. Verwende keine unnötigen Leerzeichen oder Zeilenumbrüche.

Ein Beispiel aus 11100 0,2kg * 0,192 kg zu 2,58 € = 0,50 € 160    Deutschland, Bioland Möhren wird

```csv
11100, "Möhren", "0,192kg", 2.58, 0.50, "Deutschland"
```

Du bekommst den den Input mit ### separiert.

Input: ###{PDF_INPUT}###
Output:
"""

In [149]:
loader = PyPDFLoader('attachments/Lieferschein798218.pdf')

In [150]:
fulltext = ""
for document in loader.load():
    fulltext += document.page_content

fulltext_splits = fulltext.split('\n')

header_start_index = None
header_end_index = None

for i, element in enumerate(fulltext_splits):
    if 'DE-71665Biohof Braun GbRFirma' in element.strip() and header_start_index is None:
        header_start_index = i
    if 'Steuernummer: ' in element.strip() and header_end_index is None:
        header_end_index = i

while (header_end_index is not None and header_start_index is not None):
    fulltext_splits = [fulltext_element for i, fulltext_element in enumerate(fulltext_splits) if i < header_start_index or i > header_end_index]
    header_start_index = None
    header_end_index = None
    for i, element in enumerate(fulltext_splits):
        if 'DE-71665Biohof Braun GbRFirma' in element.strip() and header_start_index is None:
            header_start_index = i
        if 'Steuernummer: ' in element.strip() and header_end_index is None:
            header_end_index = i

In [151]:
prompt = PromptTemplate(template=CONVERT_TO_CSV_TEMPLATE,
                        input_variables=["PDF_INPUT"])
llm = ChatOpenAI(model_name='gpt-3.5-turbo-0301', temperature=0)
output_parser=CommaSeparatedListOutputParser()#(pydantic_object=ParsedDocument)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)#, output_parser=output_parser)


In [152]:
print('\n'.join(fulltext_splits))

DE-ÖKO-006 Falls nicht anders vermerkt handelt es sich um Bio-Ware HK II.
"aus der Umstellung" bedeutet, dass es sich um ein Erzeugnis aus der Umstellung auf den ökologischen Landbau handelt.
Gilt nur für Lebensmittel
Hauptbestellte Menge gelieferte Menge
11100 0,3kg * 0,310 kg zu 2,40 € = 0,74 € 160    Spanien, EG-Bio Möhren
11450 0,6kg * 0,654 kg zu 3,55 € = 2,32 € 160    Spanien, EG-Bio Auberginen
11720 0,6kg * 0,744 kg zu 4,69 € = 3,49 € 160    Spanien, EG-Bio Zucchini
11434 0,6kg * 0,588 kg zu 4,09 € = 2,40 € 160    Spanien, EG-Bio Landgurken
11405 0,7kg * 0,648 kg zu 4,35 € = 2,82 € 160    Italien, EG-Bio Cherry Tomaten
11501 1Stück 1 Stück zu 1,76 € = 1,76 € 160    Eigener Anbau Biohof Braun 
BW, BiolandSchnittlauch
12127 0,42kg * 0,366 kg zu 2,75 € = 1,01 € Net    Baden-Württemberg, 
Bioland aus der UmstellungApfel der Woche
12310 0,28kg * 0,294 kg zu 3,09 € = 0,91 € Net    Ecuador, Naturland Bananen
12101 0,32kg * 0,436 kg zu 4,55 € = 1,98 € Net    Argentinien, Demeter Birnen


In [153]:
result = llm_chain(inputs={'PDF_INPUT': '\n'.join(fulltext_splits)})



> Entering new LLMChain chain...
Prompt after formatting:

Du bist ein Bot, der aus einer Einkaufsliste ein strukturiertes CSV macht.

Das CSV besteht aus unterschiedlichen Spalten:

* artikel_id
* artikel_name
* anzahl
* preis_pro_menge
* gesamtpreis
* herkunft

Außer items darf das CSV keine weiteren Elemente haben.
Es ist wichtig, dass jede gültige Position der Eingabe verarbeitet wird.
Wenn die Position nicht gültig ist, darf sie nicht in das CSV Resultat einfließen.
Gib das CSV Resultat so kompakt wie möglich aus. Verwende keine unnötigen Leerzeichen oder Zeilenumbrüche.

Ein Beispiel aus 11100 0,2kg * 0,192 kg zu 2,58 € = 0,50 € 160    Deutschland, Bioland Möhren wird

```csv
11100, "Möhren", "0,192kg", 2.58, 0.50, "Deutschland"
```

Du bekommst den den Input mit ### separiert.

Input: ###DE-ÖKO-006 Falls nicht anders vermerkt handelt es sich um Bio-Ware HK II.
"aus der Umstellung" bedeutet, dass es sich um ein Erzeugnis aus der Umstellung auf den ökologischen Landbau handelt.


In [154]:
print(*output_parser.parse(result['text']))

11100,"Möhren","0,310kg",2.40,0.74,"Spanien"
11450,"Auberginen","0,654kg",3.55,2.32,"Spanien"
11720,"Zucchini","0,744kg",4.69,3.49,"Spanien"
11434,"Landgurken","0,588kg",4.09,2.40,"Spanien"
11405,"Cherry Tomaten","0,648kg",4.35,2.82,"Italien"
11501,"Schnittlauch","1Stück",1.76,1.76,"Eigener Anbau Biohof Braun BW"
12127,"Apfel der Woche","0,366kg",2.75,1.01,"Baden-Württemberg"
12310,"Bananen","0,294kg",3.09,0.91,"Ecuador"
12101,"Birnen","0,436kg",4.55,1.98,"Argentinien"
12320,"Orangen","0,540kg",3.29,1.78,"Äthiopien"
12420,"Nektarinen gelb","0,380kg",4.99,1.90,"Italien"
24100,"Pumpernickel in der Folie 250g","1Stück",1.89,1.89,"Deutschland"
33696,"Grillkäse mediterran","1Stück",4.45,4.45,"Deutschland"
39540,"Feine weiße Bratwurst","0,398kg",19.95,7.94,"Baden-Württemberg"
23801,"Oliven Seele","2Stück",1.37,2.74,"Baden-Württemberg"
25012,"Baguette mit Zwiebeln","1Stück",2.73,2.73,"Baden-Württemberg"
26122,"MONATSSÜßES Blätte Schoko-Bana","1Stück",2.05,2.05,"Baden-Württem"
-,-,"Kistenpfand

In [155]:
from io import StringIO
import pandas as pd

TESTDATA = StringIO(''.join(output_parser.parse(result['text'])))
df = pd.read_csv(TESTDATA, names=['artikel_id', 'artikel_name', 'anzahl', 'preis_pro_menge', 'gesamtpreis', 'herkunft'])

In [156]:
from langchain.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(ChatOpenAI(temperature=0), df, verbose=True)

In [157]:
agent.run("was ist die summe des gesamtpreises?")



> Entering new AgentExecutor chain...
Thought: To find the sum of the gesamtpreis column, I can use the `sum()` function in pandas.

Action: python_repl_ast
Action Input: df['gesamtpreis'].sum()
Observation: 28.909999999999997
Thought:The sum of the gesamtpreis column is 28.91.
Final Answer: 28.91

> Finished chain.


'28.91'

In [161]:
agent.run('wie viele pfandbuchungen gab es?')



> Entering new AgentExecutor chain...
Thought: To determine the number of pfandbuchungen, I need to check if the column "herkunft" contains the word "Pfand".

Action: python_repl_ast
Action Input: df["herkunft"].str.contains("Pfand").sum()
Observation: 2
Thought:There are 2 pfandbuchungen in the dataframe.
Final Answer: 2

> Finished chain.


'2'

In [162]:
agent.run('was war am teuersten im gesamtpreis?')



> Entering new AgentExecutor chain...
Thought: To find the item with the highest total price, I need to sort the dataframe by the "gesamtpreis" column in descending order and then select the first row.

Action: python_repl_ast
Action Input: df.sort_values('gesamtpreis', ascending=False).head(1)
Observation:    artikel_id           artikel_name   anzahl  preis_pro_menge  gesamtpreis  \
13      39540  Feine weiße Bratwurst  0,398kg            19.95         7.94   

             herkunft  
13  Baden-Württemberg  
Thought:The item with the highest total price is "Feine weiße Bratwurst" from Baden-Württemberg.

Final Answer: Feine weiße Bratwurst

> Finished chain.


'Feine weiße Bratwurst'